In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
import re
from dateutil.parser import parse
from datetime import datetime
import pycountry_convert as pc

# Data Tidying and Cleaning Lab
## Reading, tidying and cleaning data. Preparing data for exploration, mining, analysis and learning

In this lab, you'll be working with the Coffee Quality Index dataset, located [here](https://www.kaggle.com/datasets/volpatto/coffee-quality-database-from-cqi). For convenience (and to save trouble in case you can't download files, or someone uploads a newer version), I've provided the dataset in the `data/` folder. The metadata (description) is at the Kaggle link. For this lab, you'll only need `merged_data_cleaned.csv`, as it is the concatenation of the other two datasets.

In this (and the following labs), you'll get several questions and problems. Do your analysis, describe it, use any tools and plots you wish, and answer. You can create any amount of cells you'd like.

Sometimes, the answers will not be unique, and they will depend on how you decide to approach and solve the problem. This is usual - we're doing science after all!

It's a good idea to save your clean dataset after all the work you've done to it.

### Problem 1. Read the dataset (1 point)
This should be self-explanatory. The first column is the index.

In [ ]:
coffee_data = pd.read_csv("data/merged_data_cleaned.csv")

In [ ]:
coffee_data

### Problem 2. Observations and features (1 point)
How many observations are there? How many features? Which features are numerical, and which are categorical?

**Note:** Think about the _meaning_, not the data types. The dataset hasn't been thoroughly cleaned.

In [ ]:
coffee_data.columns

In [ ]:
coffee_data.dtypes

In [ ]:
coffee_data["Altitude"].unique()

In [ ]:
coffee_data.Altitude.unique()

In [ ]:
coffee_data.Flavor

In [ ]:
coffee_data.Acidity

In [ ]:
plt.plot(coffee_data.Acidity)

In [ ]:
plt.hist(coffee_data.Flavor)
plt.xlim(0, 9)
plt.ylim(0, 1200)
plt.xticks(range(11))
plt.show()

### Problem 3. Column manipulation (1 point)
Make the column names more Pythonic (which helps with the quality and... aesthetics). Convert column names to `snake_case`, i.e. `species`, `country_of_origin`, `ico_number`, etc. Try to not do it manually.

In [ ]:
coffee_data.columns = (coffee_data.columns
                        .str.lower()
                        .str.replace('.', '_'))

In [ ]:
coffee_data

### Problem 4. Bag weight (1 point)
What's up with the bag weights? Make all necessary changes to the column values. Don't forget to document your methods and assumptions.

In [ ]:
coffee_data.bag_weight

### Let's save our dataset to have a backup

In [ ]:
coffee_data_backup = coffee_data

In [ ]:
coffee_data.bag_weight.unique()

In [ ]:
coffee_data.bag_weight.describe()

### Let's remove rows with 'kg,lbs', because these are unclear values.

In [ ]:
coffee_data = coffee_data[~coffee_data.bag_weight.str.contains('kg,lbs')]

In [ ]:
coffee_data.describe()

In [ ]:
coffee_data.bag_weight.unique()

### Now let's make 2 new columns - the_bag_weight and bag_weight_measurement.

In [ ]:
coffee_data["the_bag_weight"] = coffee_data.bag_weight.str.slice(0,-3)

In [ ]:
coffee_data.the_bag_weight.unique()

In [ ]:
coffee_data["bag_weight_measurement"] = coffee_data.bag_weight.str.slice(-3)

In [ ]:
coffee_data.bag_weight_measurement.unique()

In [ ]:
coffee_data.describe()

### We no longer need bag_weight column.

In [ ]:
coffee_data = coffee_data.drop(columns=["bag_weight"])

In [ ]:
coffee_data.columns

### Let's clear the_bag_weight of empty spaces

In [ ]:
coffee_data.the_bag_weight = coffee_data.the_bag_weight.str.strip()

In [ ]:
coffee_data.the_bag_weight.unique()

In [ ]:
coffee_data.describe()

### Let's clean rows with 1, 2, 6 because we don't know 1, 2, 6 kg or lbs is this

In [ ]:
coffee_data.bag_weight_measurement.unique()

In [ ]:
coffee_data = coffee_data[~coffee_data.bag_weight_measurement.str.contains('1')]

In [ ]:
coffee_data = coffee_data[~coffee_data.bag_weight_measurement.str.contains('2')]

In [ ]:
coffee_data = coffee_data[~coffee_data.bag_weight_measurement.str.contains('6')]

In [ ]:
coffee_data.describe()

In [ ]:
coffee_data.bag_weight_measurement.unique()

In [ ]:
coffee_data.shape

## Additionally we could convert lbs values to kg and round them to 2 digits

In [ ]:
def convert_lbs_to_kg(lbs_value):
    return round((lbs_value * 0.45359237), 2)

In [ ]:
coffee_data[coffee_data.bag_weight_measurement == 'lbs'].the_bag_weight.unique()

In [ ]:
coffee_data.the_bag_weight.unique()

In [ ]:
coffee_data["the_bag_weight"] = coffee_data["the_bag_weight"].astype(float)

In [ ]:
coffee_data.the_bag_weight = coffee_data.apply(
    lambda x: convert_lbs_to_kg(x["the_bag_weight"]) 
    if (x["bag_weight_measurement"] == 'lbs') 
    else x["the_bag_weight"], axis = 1)

In [ ]:
coffee_data[coffee_data.bag_weight_measurement == 'lbs'].the_bag_weight.unique()

### And now we should replace all lbs values with kg in bag_weight_measurement

In [ ]:
coffee_data.bag_weight_measurement.unique()

In [ ]:
coffee_data.loc[
    coffee_data.bag_weight_measurement == "lbs", "bag_weight_measurement"
    ] = 'kg'

In [ ]:
coffee_data.bag_weight_measurement.unique()

### Let's clean space in kg

In [ ]:
coffee_data.bag_weight_measurement = coffee_data.bag_weight_measurement.str.strip()

In [ ]:
coffee_data.bag_weight_measurement.unique()

### Now we can rename the the_bag_weight column to the_bag_weight_in_kg and remove bag_weight_measurement column.

In [ ]:
coffee_data.rename(columns={"the_bag_weight" : "the_bag_weight_in_kg"}, inplace=True)

In [ ]:
coffee_data.columns

In [ ]:
coffee_data

In [ ]:
coffee_data = coffee_data.drop(columns=["bag_weight_measurement"])

In [ ]:
coffee_data.columns

In [ ]:
coffee_data

### Problem 5. Dates (1 point)
This should remind you of problem 4 but it's slightly nastier. Fix the harvest years, document the process.

While you're here, fix the expiration dates, and grading dates. Unlike the other column, these should be dates (`pd.to_datetime()` is your friend).

### First we run unique() on the expiration and grading_date columns to find out what kind of data we have in them.

In [ ]:
coffee_data.expiration.unique()

In [ ]:
coffee_data.grading_date.unique()

In [ ]:
# def string_to_date(date_string):
 #   return datetime.strptime(date_string, "%m-%d-%Y")

In [ ]:
# from dateutil.parser import parse
# date = parse('June 21st, 2013')
# print(date)

### All our data in expiration and in grading_date columns are in format 'February 26th, 2013' - 'month_name ordered_date_number, year'. 
That's why we'll use library dateutil. The dateutil module in Python is a powerful utility for parsing and manipulating dates. When working with pandas, dateutil is automatically leveraged within the pd.to_datetime() function for parsing dates. Pandas uses the parser from the dateutil module, which can intelligently parse a wide variety of date formats.
### We define simple function which converts date_string into date.

In [ ]:
def string_to_date(date_string):
    return parse(date_string)

### We'll apply our function on expiration and grading_date.

In [ ]:
coffee_data.expiration = coffee_data.expiration.apply(string_to_date)
coffee_data.grading_date = coffee_data.grading_date.apply(string_to_date)

### Now we'll check the data in expiration and grading_date columns using unique().

In [ ]:
coffee_data.expiration.unique()

In [ ]:
coffee_data.grading_date.unique()

### Let's take a look on harvest_year column.

In [ ]:
coffee_data.harvest_year.unique()

In [ ]:
coffee_data

### Let's clean nan

In [ ]:
coffee_data.dropna(subset=['harvest_year'], inplace=True)

In [ ]:
nan_count = coffee_data['harvest_year'].isna().sum()
print (f"NAN count is {nan_count}")

In [ ]:
coffee_data['harvest_year'].count()

### There are many dummy values. Valid data will only be a 4-digit year. Let's first see how many rows we have with a valid harvest_year. Can we purge them from our dataset?

In [ ]:
count_all = coffee_data['harvest_year'].count()
print(f"All rows are {count_all}.")

In [ ]:
coffee_data['harvest_year'].unique()

In [ ]:
# let's replace any symbol "/", " ", "-" with an empty string.
# coffee_data['harvest_year'] = coffee_data['harvest_year'].str.replace(r'[-\/ ]', "", regex=True)

### Let's count the percent of rows which contain dummy harvest_year.

### Valid values are 4 digit year. Let's filter by them.

In [ ]:
pattern = r'^\d{4}$'
coffee_data_tidy = coffee_data[(coffee_data['harvest_year'].astype(str).str.match(pattern))]

In [ ]:
count_tidy = coffee_data_tidy.harvest_year.count()
count_messy = coffee_data.harvest_year.count()
percent = (count_tidy * 100 /count_messy)
print(f"Percent of rows with valid harvest year is {round(percent, 2)}")

### We have quite high percent of tidy data. That's why we'll remove these dummy data.

In [ ]:
coffee_data = coffee_data_tidy

In [ ]:
coffee_data['harvest_year'].unique()

### Let's reset the index after the transformation.

In [ ]:
coffee_data = coffee_data.reset_index(drop=True)

In [ ]:
coffee_data

### Problem 6. Countries (1 point)
How many coffees are there with unknown countries of origin? What can you do about them?

### First we analyze the unique values

In [ ]:
coffee_data.country_of_origin.unique()

### There is a nan value. Let's check how many rows have nan.

In [ ]:
coffee_data.country_of_origin.isna().sum()

### Only 1 row has value nan for country_of_origin.

### Problem 7. Owners (1 point)
There are two suspicious columns, named `Owner`, and `Owner.1` (they're likely called something different after you solved problem 3). Do something about them. Is there any link to `Producer`?

In [ ]:
coffee_data.columns

In [ ]:
coffee_data.owner.unique()

In [ ]:
coffee_data.owner_1.unique()

In [ ]:
coffee_data['owner'] = coffee_data['owner'].astype(str).str.upper()
coffee_data['owner_1'] = coffee_data['owner_1'].astype(str).str.upper()

In [ ]:
coffee_data['owner'].unique()

In [ ]:
coffee_data['owner_1'].unique()

In [ ]:
coffee_data['owner'].count()

In [ ]:
coffee_data['owner_1'].count()

### Let's find different rows

In [ ]:
def custom_compare(val1, val2):
    return val1 == val2

coffee_data['Custom_Comparison'] = coffee_data.apply(lambda row: custom_compare(row['owner'], row['owner_1']), axis=1)
print(coffee_data[coffee_data['Custom_Comparison'] == False])

### Different rows are 177 and 763. Let's analyze owner and owner_1 of these rows.

In [ ]:
print(coffee_data.iloc[177])

In [ ]:
print(coffee_data.iloc[763])

### They are actually the same. Name in column 'owner' is better written than name in 'owner_1'. That's why we'll drop 'owner_1' and 'Custom_Comparison'.

In [ ]:
coffee_data.columns

In [ ]:
coffee_data = coffee_data.drop(columns=["owner_1", "Custom_Comparison"])

In [ ]:
coffee_data.columns

### Producer is column producer.

In [ ]:
coffee_data.producer.unique()

### Problem 8. Coffee color by country and continent (1 point)
Create a table which shows how many coffees of each color are there in every country. Leave the missing values as they are.

**Note:** If you ask me, countries should be in rows, I prefer long tables much better than wide ones.

Now do the same for continents. You know what continent each country is located in.

In [ ]:
country_color_count = coffee_data.groupby(['country_of_origin', 'color']).size()

In [ ]:
country_color_count

In [ ]:
coffee_data.country_of_origin.unique()

In [ ]:
def country_to_continent(country_name):
    country_alpha2 = pc.country_name_to_country_alpha2(country_name)
    country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    return country_continent_name

country_name = 'India'
print(country_to_continent(country_name))

In [ ]:
# coffee_data["continent"] = coffee_data["country_of_origin"].apply(country_to_continent)

In [ ]:
data = {
    'country_of_origin': ['Ethiopia', 'Brazil', 'Peru', 'United States', 'China',
       'Costa Rica', 'Mexico', 'Uganda', 'Honduras', 'Taiwan',
       'Tanzania, United Republic Of', 'Kenya', 'United States (Hawaii)',
       'Thailand', 'Colombia', 'Guatemala', 'Panama', 'Papua New Guinea',
       'El Salvador', 'Indonesia', 'Nicaragua', 'Ecuador',
       'United States (Puerto Rico)', 'Haiti', 'Burundi', 'Vietnam',
       'Philippines', 'Rwanda', 'Malawi', 'Laos', 'Zambia', 'Myanmar',
       'Mauritius', 'Cote d?Ivoire', 'India'],
    'continent': ['Africa', 'South America', 'South America', 'North America', 'Asia',
       'North America', 'North America', 'Africa', 'North America', 'Asia',
       'Africa', 'Africa', 'North America',
       'Asia', 'South America', 'North America', 'North America', 'Oceania',
       'North America', 'Asia', 'North America', 'South America',
       'North America', 'North America', 'Africa', 'Asia',
       'Asia', 'Africa', 'Africa', 'Asia', 'Africa', 'Asia',
       'Africa', 'Africa', 'Asia']
}

# Create DataFrame from the dictionary
continent_df = pd.DataFrame(data)

In [ ]:
# Merge the DataFrames
merged_df = pd.merge(coffee_data, continent_df, on='country_of_origin')

In [ ]:
# Display the merged DataFrame
print(merged_df)

In [ ]:
continent_color_count = merged_df.groupby(['continent', 'color']).size()
continent_color_count

### Problem 9. Ratings (1 point)
The columns `Aroma`, `Flavor`, etc., up to `Moisture` represent subjective ratings. Explore them. Show the means and range; draw histograms and / or boxplots as needed. You can even try correlations if you want. What's up with all those ratings?

### Problem 10. High-level errors (1 point)
Check the countries against region names, altitudes, and companies. Are there any discrepancies (e.g. human errors, like a region not matching the country)? Take a look at the (cleaned) altitudes; there has been a lot of preprocessing done to them. Was it done correctly?

### * Problem 11. Clean and explore at will
The dataset claimed to be clean, but we were able to discover a lot of things to fix and do better.

Play around with the data as much as you wish, and if you find variables to tidy up and clean - by all means, do that!